# Загрузка Pandas и очистка данных

In [464]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import seaborn as sns
# Загружаем специальный инструмент для разбивки:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor # инструмент для создания и обучения модели
from sklearn import metrics # инструменты для оценки точности модели

In [465]:
df = pd.read_csv('main_task_new.csv')

In [466]:
df.sample(5)

,Restaurant_id,City,Cuisine Style,Ranking,Rating,Price Range,Number of Reviews,Reviews,URL_TA,ID_TA
17669,id_3697,Prague,NaN,3706.0,3.0,NaN,NaN,"[['Not excellent but not bad'], ['05/09/2014']]",/Restaurant_Review-g274707-d4768616-Reviews-No...,d4768616
24190,id_2899,Rome,['Pizza'],2900.0,4.5,NaN,65.0,"[[], []]",/Restaurant_Review-g187791-d3701368-Reviews-Er...,d3701368
25465,id_1148,Prague,"['French', 'Bar', 'Cafe', 'International', 'Eu...",1149.0,4.0,$$ - $$$,92.0,"[['Good Service', 'Cosy Cafe'], ['06/15/2017',...",/Restaurant_Review-g274707-d3172080-Reviews-Ca...,d3172080
4366,id_978,Rome,"['Italian', 'Pizza', 'Mediterranean', 'Europea...",979.0,4.0,$$ - $$$,413.0,"[['Great food great service', 'Local and authe...",/Restaurant_Review-g187791-d8299095-Reviews-Ma...,d8299095
3888,id_2051,Hamburg,NaN,2055.0,4.0,NaN,NaN,"[[], []]",/Restaurant_Review-g187331-d8537733-Reviews-Tu...,d8537733


#### основные поля

- Restaurant_id — идентификационный номер ресторана / сети ресторанов;
- City — город, в котором находится ресторан;
- Cuisine Style — кухня или кухни, к которым можно отнести блюда, предлагаемые в ресторане;
- Ranking — место, которое занимает данный ресторан среди всех ресторанов своего города;
- Rating — рейтинг ресторана по данным TripAdvisor (именно это значение должна будет предсказывать модель);
- Price Range — диапазон цен в ресторане;
- Number of Reviews — количество отзывов о ресторане;
- Reviews — данные о двух отзывах, которые отображаются на сайте ресторана;
- URL_TA — URL страницы ресторана на TripAdvisor;
- ID_TA — идентификатор ресторана в базе данных TripAdvisor.

In [32]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40000 entries, 0 to 39999
Data columns (total 13 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Restaurant_id      40000 non-null  object 
 1   City               40000 non-null  object 
 2   Cuisine Style      30717 non-null  object 
 3   Ranking            40000 non-null  float64
 4   Rating             40000 non-null  float64
 5   Price Range        26114 non-null  object 
 6   Number of Reviews  37457 non-null  float64
 7   Reviews            40000 non-null  object 
 8   URL_TA             40000 non-null  object 
 9   ID_TA              40000 non-null  object 
 10  cuisine_count      40000 non-null  int64  
 11  reviews_dates      33529 non-null  object 
 12  days_between       28973 non-null  float64
dtypes: float64(4), int64(1), object(8)
memory usage: 4.0+ MB


In [169]:
def missing_values_table(df):
    
        mis_val = df.isnull().sum()
        mis_val_percent = 100 * df.isnull().sum() / len(df)
        mis_val_table = pd.concat([mis_val, mis_val_percent], axis=1)
        mis_val_table_ren_columns = mis_val_table.rename(
        columns = {0 : 'Отсутствующих данных', 1 : '% от всех данных'})
        
        mis_val_table_ren_columns = mis_val_table_ren_columns[
            mis_val_table_ren_columns.iloc[:,1] != 0].sort_values(
        '% от всех данных', ascending=False).round(1)
        
        print ("Было выбрано " + str(df.shape[1]) + " колонок.\n"      
            "Среди их " + str(mis_val_table_ren_columns.shape[0]) +
              " с отсутствующими данными.")
        
        return mis_val_table_ren_columns
    
missing_values_table(df)

Было выбрано 10 колонок.
Среди их 3 с отсутствующими данными.


,Отсутствующих данных,% от всех данных
Price Range,13886,34.7
Cuisine Style,9283,23.2
Number of Reviews,2543,6.4


In [50]:
# код по очистке данных и генерации новых признаков

In [467]:
# кол-во видов кухни
def cuisine_count(value):
    if not pd.isna(value):
        return len(value.split(','))
    else:
        return 1

df['cuisine_count'] = df['Cuisine Style'].apply(cuisine_count)

In [468]:
# текст отзывов
def reviews_text(value):
    review = value.replace("'",'').strip('[]').split('], [')
    if len(review) > 1:
        text = review[0]
        return text
    else:
        return np.nan
    
df['reviews_text'] = df['Reviews'].apply(reviews_text)

In [469]:
# даты отзывов
def reviews_date(value):
    review = value.replace("'",'').strip('[]').split('], [')
    if len(review) > 1:
        dates = review[1].replace(' ', '')
        return dates
    else:
        return np.nan
    
df['reviews_dates'] = df['Reviews'].apply(reviews_date)

In [470]:
# кол-во дней между отзывами
def days_between(dates):
    if pd.isna(dates):
        return 0
    else:
        date_list = dates.split(',')
        if len(date_list) > 1:  
            d1 = datetime.strptime(date_list[0], '%m/%d/%Y')
            d2 = datetime.strptime(date_list[1], '%m/%d/%Y')
            return abs((d2 - d1).days)
        else:
            return 0
    
df['days_between'] = df['reviews_dates'].apply(days_between)

In [471]:
# наличие отзывов
def reviews_pres(review):
    if pd.isna(review):
        return 0
    else:
        return 1
    
df['reviews_pres'] = df['reviews_text'].apply(reviews_pres)

In [472]:
# уровень цен
def price_level(value):
    if pd.isna(value):
        return 2
    else:
        if value == '$':
            return 1
        elif value == '$$ - $$$':
            return 2
        elif value == '$$$$':
            return 3
    
df['price_level'] = df['Price Range'].apply(price_level)

In [473]:
# редактирование поля 'Number of Reviews'
def number_rev_ed(value):
    if pd.isna(value[0]):
        if pd.isna(value[1]):
            return 0
        else:
            return len(value[1].split(','))
    else:
        return value[0]
        
df['Number of Reviews'] = df[['Number of Reviews', 'reviews_dates']].apply(number_rev_ed, axis=1)    

In [474]:
# Ranking/Number of Reviews
def ranking_reviews(value):
    if value[1] > 0:
        return value[0] / value[1]
    else:
        return 0

df['ranking_reviews'] = df[['Ranking', 'Number of Reviews']].apply(ranking_reviews, axis=1)  

In [475]:
# площадь городов
city_area_dict = {
    'London': 1572,
    'Paris': 105.4,
    'Madrid': 604.3,
    'Barcelona': 101.9,
    'Berlin': 891.8,
    'Milan': 181.8,
    'Rome': 1285,
    'Prague': 496,
    'Lisbon': 100,
    'Vienna': 414.6,
    'Amsterdam': 219.3,
    'Brussels': 32.61,
    'Hamburg': 755.2,
    'Munich': 310.7,
    'Lyon': 47.87,
    'Stockholm': 188,
    'Budapest': 525.2,
    'Warsaw': 517.2,
    'Dublin': 117.8,
    'Copenhagen': 88.25,
    'Athens': 38.96,
    'Edinburgh': 264,
    'Zurich': 87.88,
    'Oporto': 41.42,
    'Geneva': 15.93,
    'Krakow': 327,
    'Oslo': 454,
    'Helsinki': 213.8,
    'Bratislava': 367.6,
    'Luxembourg': 51.46,
    'Ljubljana': 163.8,
}

def city_area(value):
    return city_area_dict.get(value)

df['city_area'] = df['City'].apply(city_area)

In [476]:
# население городов
city_population_dict = {
    'London': 8.982,
    'Paris': 2.161,
    'Madrid': 3.223,
    'Barcelona': 1.62 ,
    'Berlin': 3.645 ,
    'Milan': 1.352 ,
    'Rome': 2.873 ,
    'Prague': 1.309 ,
    'Lisbon': 0.505,
    'Vienna': 1.897,
    'Amsterdam': 0.822,
    'Brussels': 0.174,
    'Hamburg': 1.841 ,
    'Munich': 1.472 ,
    'Lyon': 0.513,
    'Stockholm': 0.975,
    'Budapest': 1.756 ,
    'Warsaw': 1.765 ,
    'Dublin': 0.544,
    'Copenhagen': 0.602,
    'Athens': 0.664,
    'Edinburgh': 0.482,
    'Zurich': 0.403,
    'Oporto': 0.214,
    'Geneva': 0.199,
    'Krakow': 0.766,
    'Oslo': 0.634,
    'Helsinki': 0.631,
    'Bratislava': 0.424,
    'Luxembourg': 0.124,
    'Ljubljana': 0.279,
}

def city_population(value):
    return city_population_dict.get(value)

df['city_population'] = df['City'].apply(city_area)

### Корреляция

In [477]:
corr = df.corr()
cmap = sns.diverging_palette(5, 250, as_cmap=True)

def magnify():
    return [dict(selector="th",
                 props=[("font-size", "7pt")]),
            dict(selector="td",
                 props=[('padding', "0em 0em")]),
            dict(selector="th:hover",
                 props=[("font-size", "12pt")]),
            dict(selector="tr:hover td:hover",
                 props=[('max-width', '200px'),
                        ('font-size', '12pt')])
]

corr.style.background_gradient(cmap, axis=1)\
    .set_properties(**{'max-width': '80px', 'font-size': '10pt'})\
    .set_caption("Hover to magify")\
    .set_precision(2)\
    .set_table_styles(magnify())

,Ranking,Rating,Number of Reviews,cuisine_count,days_between,reviews_pres,price_level,ranking_reviews,city_area,city_population
Ranking,1.00,-0.37,-0.22,-0.32,0.07,-0.11,-0.03,0.51,0.39,0.39
Rating,-0.37,1.00,0.03,0.12,-0.07,0.02,-0.03,-0.11,0.02,0.02
Number of Reviews,-0.22,0.03,1.00,0.41,-0.09,0.16,0.11,-0.14,0.05,0.05
cuisine_count,-0.32,0.12,0.41,1.00,-0.12,0.29,0.00,-0.25,0.08,0.08
days_between,0.07,-0.07,-0.09,-0.12,1.00,0.23,-0.04,-0.03,-0.02,-0.02
reviews_pres,-0.11,0.02,0.16,0.29,0.23,1.00,-0.08,-0.07,0.10,0.10
price_level,-0.03,-0.03,0.11,0.00,-0.04,-0.08,1.00,0.05,-0.04,-0.04
ranking_reviews,0.51,-0.11,-0.14,-0.25,-0.03,-0.07,0.05,1.00,0.23,0.23
city_area,0.39,0.02,0.05,0.08,-0.02,0.10,-0.04,0.23,1.00,1.00
city_population,0.39,0.02,0.05,0.08,-0.02,0.10,-0.04,0.23,1.00,1.00


#### dummy - переменные

In [478]:
# dummy - переменные
city_dum = pd.get_dummies(df['City'])
df = df.join(city_dum)

### Удаление стобцов

In [479]:
drop_col = ['City', 'Cuisine Style',
            'Price Range', 'Reviews',
            'URL_TA', 'ID_TA',
            'reviews_text',
            'reviews_dates',]
df = df.drop(columns=drop_col, axis=1)

# Разбиваем датафрейм на части, необходимые для обучения и тестирования модели

In [480]:
# Х - данные с информацией о ресторанах, у - целевая переменная (рейтинги ресторанов)
X = df.drop(['Restaurant_id', 'Rating'], axis = 1)
y = df['Rating']

In [481]:
# Наборы данных с меткой "train" будут использоваться для обучения модели, "test" - для тестирования.
# Для тестирования мы будем использовать 25% от исходного датасета.
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25)

# Создаём, обучаем и тестируем модель

In [484]:
# Создаём модель
regr = RandomForestRegressor(n_estimators=100)

# Обучаем модель на тестовом наборе данных
regr.fit(X_train, y_train)

# Используем обученную модель для предсказания рейтинга ресторанов в тестовой выборке.
# Предсказанные значения записываем в переменную y_pred
y_pred = regr.predict(X_test)

In [485]:
# Сравниваем предсказанные значения (y_pred) с реальными (y_test), и смотрим насколько они в среднем отличаются
# Метрика называется Mean Absolute Error (MAE) и показывает среднее отклонение предсказанных значений от фактических.
print('MAE:', metrics.mean_absolute_error(y_test, y_pred))

MAE: 0.21463249999999998
